# Chap 06 파이썬 고급 주제

# 6.1 멀티 프로세스와 멀티 스레드
<hr>

프로그램의 작업을 동시에 수행하는 방법은 멀티 프로세스와 멀티 스레드이다.

- **멀티 프로세스**
    * 별도의 메모리 영역을 가지며, 특별한 메커니즘으로만 통신 가능
    * 프로세스 간 데이터 공유와 통신용으로는 비효율적
    * 파이썬에서는 subprocess 모듈을 사용
    
- **멀티 스레드**
    * 단일 프로세스 내의 멀티 스레드는 동일한 메모리에 접근
    * threading 모듈의 처리를 ㅗㅇ해 한번에 한 스레드만 메모리 영역에 접근 가능
    * 각 프로세스가 독립적인 stack, heap, code, data 영역을 가지는 반면, 한 프로세스에 속한 스레드는 stack 영역을 제외한 메모리 영역을 공유
    
파이썬에 스레드 매커니즘이 있긴 하나, 진정한 병렬 실행이 지원되는 것은 아님. 하지만 오늘날 운영체제에서 충분히 효율적임.

## 6.1.1 subprocess 모듈

- subprocess 모듈은 '부모-자식' 프로세스 쌍을 생성하는 데 사용
- 부모 프로세스는 차례로 다른 일을 처리하는 자식 프로세스의 인스턴스를 실행
- 자식 프로세스를 사용함으로써 멀티 코어의 이점을 최대한 취하고, 동시성(concurrency) 문제를 운영체제가 알아서 처리하도록 한다

## 6.1.2 threading 모듈

- 스레드가 여러개로 분리되면, 스레드 간 데이터 고융의 복잡성이 증가
- 또한 lock과 deadlock을 회피하는 데 주의를 기울여야 함
- 파이썬에서는 단 하나의 메인 스레드만 존재
- 멀티 스레드를 사용하려면 threading 모듈 사용

- lock 관리를 위해 queue 모듈 사용
- 큐에 의존하면 자원의 접근을 직렬화할 수 있고, 이는 곧 한번에 하나의 스레드만 데이터에 접근할 수 있게 한다는 뜻(FIFO)

- worker thread가 작업을 완료했는데도 프로그램이 종료되지 않고 계속 실행되는 경우 문제가 될 수 있다
- 스레드를 데몬으로 변환하면 데몬 스레드가 실행되지 않는 즉시 프로그램이 종료
- queue.join() 메서드는 큐가 빌 때까지(큐의 모든 항목이 처리될 때까지) 기다린다

In [ ]:
import queue
import threading

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        # 작업을 처리한다.
        print("스레드 {0} : 처리 완료 {1}".format(num+1, item))
        q.task_done()
        
if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)
        
    for item in range(20):
        q.put(item)
        
    # 모든 작업이 끝날 때까지 대기한다(block).
    q.join()
    
    #워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

## 6.1.3 뮤텍스와 세마포어

### 뮤텍스
- 공유 리소스에 한번에 하나의 스레드만 접근 할 수 있도록 하는 상호 배제(mutual exclusion) 동시성 제어 정책을 강제하기 위해 설계
- 예를 들어 한 스레드가 배열을 수정하고 있다고 가정. 배열 작업을 절반 이상 수행했을 때, 프로세서가 다른 스레드로 전환한다면 두 스레드가 동시에 배열을 수정하는 일이 발생함
- 개념적으로 뮤텍스는 1부터 시작하는 정수
- 스레드는 배열을 변경해야 할 때마다 뮤텍스를 '잠근다'. 즉, 스레드는 뮤텍스가 양수가 될 때까지 대기한 다음 숫자를 1 감소시킨다(이것이 곧 락). 배열 수정을 마치면 뮤텍스가 잠금 해제되어 숫자가 1 증가(언락)
- 배열 수정 전 뮤텍스를 잠근 후, 수정 작업이 끝나고 잠금을 해제하면, 두 스레드가 배열을 동시에 수정하는 일은 일어나지 않음

참조: 2_threading_mutex.py

### 세마포어(semaphore)
- 뮤텍스보다 더 일반적으로 사용되는 개념
- 세마포어는 1보다 큰 수로 시작 가능
- 세마포어 값은 곧 한번에 자원에 접근할 수 있는 스레드의 수
- 세마포어는 뮤텍스의 락 및 언락 작업과 유사한 대기(wait) 및 신호(signal) 작업을 지원
- 파이썬의 뮤텍스(락)과 세마포어에 관한 내용은 threading 모듈의 공식 문서를 참조

참조: 3_threading_semaphore.py

## 6.1.4 데드락과 스핀락

- 데드락(교착상태, deadlock)은 두개 이상의 프로세스나 스레드가 서로 상대방의 작업이 끝나기만을 기다리고 있기 때문에 결과적으로 아무것도 완료되지 못하는 상태
- 아래 네가지 조건을 모두 충족하면 데드락이 발생. 네가지 조건 중 하나라도 막을 수 있다면, 데드락 해결
    * 상호 배제(mutual exclusion): 자원은 한번에 한 프로세스(혹은 스레드)만 사용할 수 있다
    * 점유와 대기(hold and wait): 한 프로세스가 자원을 가지고 있는 상태에서, 다른 프로세스가 쓰는 자원의 반납을 기다림
    * 비선점(no preemption): 다른 프로세스가 이미 점유한 자원을 강제로 뺏어오지 못함
    * 순환대기(circular wait): 프로세스 A, B, C가 있다고 가정할 때 A는 B가 점유한 자원을, B는 C가 점유한 자원을, C는 A가 점유한 자원을 대기하는 상태
    
- 스핀락(spinlock)은 고성능 컴퓨팅 상황에 유용한 바쁜 대기(busy waiting)의 한 형태. 스핀락은 임계 구역에 진입이 불가능할 때, 진입이 가능할 때까지 반복문을 돌면서 재시도하는 방식으로 구현된 락

## 6.1.5 스레딩에 대한 구글 파이썬 스타일 가이드

내장 타입의 원자성(atomicity)에 의존하지 않는다. 딕셔너리 같은 파이썬 기본 데이터 타입은 원자적 연산을 수행하는 반면, 내장 타입이 원자적이지 않은 경우가 있어서(\_\_hash\_\_() 또는 \_\_eq\_\_() 메서드가 구현된 경우), 내장 타입의 원자성에 의존해선 안됨. 또한 원자적 변수 할당에 의존하지 않아야 함.

queue 모듈의 Queue 데이터 타입을 스레드 간 데이터를 전달하는 기본 방식으로 사용. 그렇지 않으면 threading 모듈의 락을 사용.

- threading.Condition.wait()은 돌고있는 스레드를 잠시 멈추게 하고, threading.Condition.notifyAll()은 잠시 멈춘 스레드를 다시 동작하게 한다.

In [1]:
import threading

def consumer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 대기".format(name))
        cond.wait()
        print("{0} 자원 소비".format(name))

def producer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()


if __name__ == "__main__":
    condition = threading.Condition()
    consumer1 = threading.Thread(
        name="소비자1", target=consumer, args=(condition,)
    )
    consumer2 = threading.Thread(
        name="소비자2", target=consumer, args=(condition,)
    )
    producer = threading.Thread(
        name="생산자", target=producer, args=(condition,)
    )

    consumer1.start()
    consumer2.start()
    producer.start()


소비자1 시작소비자2 시작
소비자2 대기

소비자1 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자1 자원 소비
소비자2 자원 소비


# 6.2 좋은 습관
<hr>

## 6.2.1 가상환경

### virtualenv
- virtualenv는 각 파이썬 프로젝트별로 필요한 패키지를 따로 관리하고자 만들어진 모듈
- 실행 방법은 아래와 같다.

```
# virtualenv 설치
$ pip install virtualenv

# 설치된 버전 확인
$ virtualenv --version

# 가상 환경 프로젝트 생성
$ cd myvenv
$ virtualenv myvenv

# 가상 환경 프로젝트 활성화
$ source myvenv\Scripts\activate

# 파이썬 외부 패키지 모듈 설치
(myvenv)$ pip install requests

# 가상 환경에서 설치된 외부 패키지 목록 확인
(myvenv)$ pip freeze

# 가상 환경 프로젝트 비활성화
(myvenv)$ deactivate
```

### virtualenvwrapper
- virtualenvwrapper는 virtualenv를 사용하여 모든 가상 환경을 한곳에 배치
- https://pypi.org/project/virtualenvwrapper-win/ (윈도우용)

```
# virtualenvwrapper 설치
pip install virtualenvwrapper-win

> mkvirtualenv를 입력하여 "Pass a name to create a new virtualenv"가 뜨면 성공

# 가상환경 폴더 생성
setx WORKON_HOME 원하는_디렉토리_위치

# 가상환경 생성(프롬프트 창 닫고 재실행해야 함)
mkvirtualenv 원하는_가상환경_이름

(원하는_가상환경_이름) \>

# deactivate 커멘드로 가상환경에서 빠져나오기
(원하는_가상환경_이름) \> deactivate

# workon 커멘드로 가상환경에 진입
workon 원하는_가상환경_이름

(원하는_가상환경_이름) \>

# 원하는 프로그램 (ex 장고) 설치
(원하는_가상환경_이름) \> pip install django
```

## 6.2.2 디버깅

- 파이썬 공식문서 참조(https://docs.python.org/3/library/pdb.html)
- 대화식 인터피리터를 사용해 코드를 살펴보고 싶다면 -i 삽입(python -i 1_threading_with_queue.py)
- 혹은 -m pdb 삽입(python -m pdb 1_threading_with_queue.py)

```
(base) C:\05.Python\Algorithms\Chap06>python -m pdb 1_threading_with_queue.py
> c:\05.python\algorithms\chap06\1_threading_with_queue.py(1)<module>()
-> import queue
(Pdb) help

Documented commands (type help <topic>):
========================================
EOF    c          d        h         list      q        rv       undisplay
a      cl         debug    help      ll        quit     s        unt
alias  clear      disable  ignore    longlist  r        source   until
args   commands   display  interact  n         restart  step     up
b      condition  down     j         next      return   tbreak   w
break  cont       enable   jump      p         retval   u        whatis
bt     continue   exit     l         pp        run      unalias  where

Miscellaneous help topics:
==========================
exec  pdb

(Pdb) help n
n(ext)
        Continue execution until the next line in the current function
        is reached or it returns.
(Pdb) n
> c:\05.python\algorithms\chap06\1_threading_with_queue.py(2)<module>()
-> import threading
(Pdb) n
> c:\05.python\algorithms\chap06\1_threading_with_queue.py(4)<module>()
-> q = queue.Queue()
(Pdb) n
> c:\05.python\algorithms\chap06\1_threading_with_queue.py(6)<module>()
-> def worker(num):
(Pdb) quit()
```

## 6.2.3 프로파일링

프로그램이 매우 느리게 실행되거나 예상보다 많은 메모리가 소비된다면, 자료구조나 알고리즘을 잘못 선택했거나 비효율적으로 구현했기 때문인 경우가 많다.

이 경우 다음과 같이 성능 항목을 검토한다.
- 읽기 전용 데이터는 리스트 대신 튜플 사용
- 반복문에서 항목이 많은 리스트나 튜플 대신 **제너레이터**를 사용하여 순회
- 문자열 연결시 \+ 연산자로 문자열을 연결하는 대신, 리스트에 문자열 추가(append) 후, 마지막에 join

다음 구글 파이썬 스타일 가이드의 예제 코드를 살펴보자.

```
# 좋은 예
items = ['<table>']
for last_name, first_name in employee_list:
    items.append('<tr><td>%s, %s</td></tr>' % (last_name, first_name))
items.append('</table>')
employee_table = ''.join(items)

# 나쁜 예
employee_table = '<table>'
for last_name, first_name in employee_list:
    employee_table += '<tr><td>%s, %s</td></tr>' % (last_name, first_name)
employee_table += '</table>'

```

### cProfile 모듈

cProfile 모듈은 호출 시간에 대한 세부 분석을 제공하며, 병목 현상을 찾는 데 사용

In [1]:
import cProfile
import time

def sleep():
    time.sleep(5)
    
def hello_world():
    print("Hello World!")
    
def main():
    sleep()
    hello_world()
    
cProfile.run('main()')

Hello World!
         42 function calls in 5.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.001    5.001 <ipython-input-1-94844ef2a241>:10(main)
        1    0.000    0.000    5.000    5.000 <ipython-input-1-94844ef2a241>:4(sleep)
        1    0.000    0.000    0.001    0.001 <ipython-input-1-94844ef2a241>:7(hello_world)
        1    0.000    0.000    5.001    5.001 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        2    0.000    0.000    0.001    0.000 iostream.py:382(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:334(send)
        3    0.000    0.000    0.000    0.000 threading.py:1050(_wa

혹은 다음과 같이 스크립트 파일에 대해 실행 가능

```
(base) C:\05.Python\Algorithms\Chap06>python -m cProfile -o profile.dat 1_threading_with_queue.py
스레드 1 : 처리 완료 0
스레드 2 : 처리 완료 1
스레드 4 : 처리 완료 3
스레드 1 : 처리 완료 5
스레드 2 : 처리 완료 6
스레드 4 : 처리 완료 7
스레드 1 : 처리 완료 8
스레드 2 : 처리 완료 9
스레드 4 : 처리 완료 10
스레드 1 : 처리 완료 11
스레드 2 : 처리 완료 12
스레드 4 : 처리 완료 13
스레드 1 : 처리 완료 14
스레드 2 : 처리 완료 15
스레드 4 : 처리 완료 16
스레드 1 : 처리 완료 17
스레드 2 : 처리 완료 18
스레드 4 : 처리 완료 19
스레드 5 : 처리 완료 4
스레드 3 : 처리 완료 2

(base) C:\05.Python\Algorithms\Chap06>python -m pstats profile.dat
Welcome to the profile statistics browser.
profile.dat% help

Documented commands (type help <topic>):
========================================
EOF  add  callees  callers  help  quit  read  reverse  sort  stats  strip

profile.dat% help callees
Print callees statistics from the current stat object.
Arguments may be:
* An integer maximum number of entries to print.
* A decimal fractional number between 0 and 1, controlling
  what fraction of selected entries to print.
* A regular expression; only entries with function names
  that match it are printed.
profile.dat% help callers
Print callers statistics from the current stat object.
Arguments may be:
* An integer maximum number of entries to print.
* A decimal fractional number between 0 and 1, controlling
  what fraction of selected entries to print.
* A regular expression; only entries with function names
  that match it are printed.
profile.dat% callees
   Random listing order was used

Function                                                              called...
                                                                          ncalls  tottime  cumtime
<frozen importlib._bootstrap>:35(_new_module)                         ->
<frozen importlib._bootstrap>:58(__init__)                            ->      14    0.000    0.000  {built-in method _thread.allocate_lock}
<frozen importlib._bootstrap>:78(acquire)                             ->       7    0.000    0.000  {built-in method _thread.get_ident}
<frozen importlib._bootstrap>:103(release)                            ->       7    0.000    0.000  {built-in method _thread.get_ident}
<frozen importlib._bootstrap_external>:36(_relax_case)                ->
<frozen importlib._bootstrap>:143(__init__)                           ->
<frozen importlib._bootstrap>:147(__enter__)                          ->       7    0.000    0.000  <frozen importlib._bootstrap>:78(acquire)
                                                                               7    0.000    0.000  <frozen importlib._bootstrap>:157(_get_module_lock)
<frozen importlib._bootstrap>:151(__exit__)                           ->       7    0.000    0.000  <frozen importlib._bootstrap>:103(release)
<frozen importlib._bootstrap>:176(cb)                                 ->       7    0.000    0.000  {built-in method _imp.acquire_lock}
                                                                               7    0.000    0.000  {built-in method _imp.release_lock}
                                                                               7    0.000    0.000  {method 'get' of 'dict' objects}
<frozen importlib._bootstrap>:157(_get_module_lock)                   ->       7    0.000    0.000  <frozen importlib._bootstrap>:58(__init__)
                                                                               7    0.000    0.000  {built-in method _imp.acquire_lock}
                                                                               7    0.000    0.000  {built-in method _imp.release_lock}
<frozen importlib._bootstrap>:211(_call_with_frames_removed)          ->       1    0.005    0.005  {built-in method _imp.create_dynamic}
                                                                               1    0.000    0.000  {built-in method _imp.exec_dynamic}
                                                                             6/1    0.000    0.026  {built-in method builtins.exec}
```

### timeit 모듈

코드 일부분의 실행 시간을 확인하는 데 사용

In [2]:
import timeit
timeit.timeit("x = 2+2")

0.033101199999691744

In [3]:
import timeit
timeit.timeit("x = sum(range(10))")

0.61006610000004

다음과 같이 스크립트로도 실행 가능

```
(base) C:\05.Python\Algorithms\Chap06>python -m timeit "d = {}"
5000000 loops, best of 5: 45 nsec per loop

(base) C:\05.Python\Algorithms\Chap06>python -m timeit "import collections" "d = collections.OrderedDict()"
1000000 loops, best of 5: 337 nsec per loop
```

In [4]:
# 5_using_time_module.py
import time
def sumOfN2(n):
    start = time.time()
    theSum = 0
    for i in range(1, n+1):
        theSum = theSum + i
    end = time.time()
    return theSum, end-start


if __name__ == "__main__":
    n = 5
    print("총 합계: %d\t 시간: %10.7f초" % sumOfN2(n))
    n = 200
    print("총 합계: %d\t 시간: %10.7f초" % sumOfN2(n))

총 합계: 15	 시간:  0.0000000초
총 합계: 20100	 시간:  0.0000000초


# 6.3 단위 테스트
<hr>

- 개별 함수 및 클래스의 메서드에 대한 테스트 코드를 작성하여 예상한 값이 맞게 나오는지 확인하는 것이 좋은 습관임
- 파이썬 표준 라이브러리에는 doctest와 unittest 모듈을 제공
- 외부 라이브러리인 pytest 모듈도 존재

## 6.3.1 용어
- test fixture: 테스트 설정을 위한 코드(테스트용 입력 파일을 만들었다 삭제하는 코드)
- test case: 테스트의 기본 단위
- test suite: unittest.TestCase의 하위 클래스에 의해 생성된 테스트 케이스 집합. 각 테스트 케이스의 메서드 이름은 test로 시작
- test runner: 하나 이상의 테스트 스위트를 실행하는 객체

## 6.3.2 doctest
- doctest 모듈은 모듈과 함수의 docstring 안에 테스트 코드를 작성할 때 사용. 테스트를 작성한 후 다음 코드 세줄만 추가하면 된다.

```
if __name__ = "__main__:
    import doctest
    doctest.testmod()
```

doctest 모듈이 포함된 프로그램은 두가지 방법으로 실행 가능
- 먼저 -v 옵션으로 파이썬을 실행

In [6]:
"""
>>> factorial(6)
720

>>> [factorial(n) for n in range(6)]
[1, 1, 2, 6, 24, 120]

>>> factorial(30)
265252859812191058636308480000000

"""

def factorial(n):
    import math
    if not n >= 0:
        raise ValueError("n must be >= 0")
    if math.floor(n) != n:
        raise ValueError("n must be exact integer")
    if n+1 == n:  # catch a value like 1e300
        raise OverflowError("n too large")
    
    result = 1
    factor = 2

    while factor <= n :
        result *= factor
        factor += 1
    
    return result

if __name__ == "__main__":
    import doctest
    doctest.testmod()
    

```
(base) C:\05.Python\Algorithms\Chap06>python 6_doctest_factorial.py -v
Trying:
    factorial(6)
Expecting:
    720
ok
Trying:
    [factorial(n) for n in range(6)]
Expecting:
    [1, 1, 2, 6, 24, 120]
ok
Trying:
    factorial(30)
Expecting:
    265252859812191058636308480000000
ok
1 items had no tests:
    __main__.factorial
1 items passed all tests:
   3 tests in __main__
3 tests in 2 items.
3 passed and 0 failed.
Test passed.
```

## 6.3.3 pytest

- pytest는 사용법이 매우 쉽다. 
- test로 시작하는 파일에서 test로 시작하는 함수를 작성하기만 하면 된다
- 간단한 예를 살펴보자

먼저 pytest 라이브러리 설치
```
(base) C:\05.Python\Algorithms\Chap06>pip install pytest
```

test_pytest.py 파일 생성
```
# test_pytest.py
def func(x):
    return x + 1

def test_answer():
    assert func(3) == 51
```

터미널에서 다음 명령 실행하면, 파일명이 test로 시작하는 파이썬 스크립트 실행
```
(base) C:\05.Python\Algorithms\Chap06>python -m pytest
============================= test session starts =============================
platform win32 -- Python 3.7.7, pytest-3.8.0, py-1.6.0, pluggy-0.7.1
rootdir: C:\05.Python\Algorithms\Chap06, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item

test_pytest.py F                                                         [100%]

================================== FAILURES ===================================
_________________________________ test_answer _________________________________

    def test_answer():
>       assert func(3) == 51
E       assert 4 == 51
E        +  where 4 = func(3)

test_pytest.py:5: AssertionError
========================== 1 failed in 0.24 seconds ===========================
```

아래와 같이 파일을 지정하여 실행도 가능
```
(base) C:\05.Python\Algorithms\Chap06>py.test test_pytest.py
============================= test session starts =============================
platform win32 -- Python 3.7.7, pytest-3.8.0, py-1.6.0, pluggy-0.7.1
rootdir: C:\05.Python\Algorithms\Chap06, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item

test_pytest.py F                                                         [100%]

================================== FAILURES ===================================
_________________________________ test_answer _________________________________

    def test_answer():
>       assert func(3) == 51
E       assert 4 == 51
E        +  where 4 = func(3)

test_pytest.py:5: AssertionError
========================== 1 failed in 0.20 seconds ===========================
```